In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import numpy as np # linear algebra
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import random


In [ ]:
debug = True
CONF_THRE = 0.3
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')

# used from https://www.kaggle.com/go5kuramubon/merge-label-and-tracking-data?select=train_label_tracking_merged.csv
merged_train_labels = pd.read_csv('../input/merged/train_label_tracking_merged.csv')

In [ ]:
%pylab inline 
import cv2
from IPython.display import clear_output

In [ ]:
merged_train_labels.sort_values(by=['video','frame']).head(8498)

In [ ]:
merged_sorted_data = np.array(merged_train_labels.sort_values(by=['video','frame']))


In [ ]:
#reorganize data for visualization
# split dataset to sideline and endline

merged_sorted_reorganized_data_Endzone = []
merged_sorted_reorganized_data_Sideline = []
tmp_array = []
frame_counter = 0

for k in range(len(merged_sorted_data)):
    if str(merged_sorted_data[k][3]) == "Endzone":
        if merged_sorted_data[k][5] > frame_counter :
            if frame_counter > 0:
                merged_sorted_reorganized_data_Endzone.append(tmp_array)
            tmp_array = []
            tmp_array.append(merged_sorted_data[k][0])
        tmp_array.append(np.concatenate((merged_sorted_data[k][5:10],merged_sorted_data[k][15:22])))
        frame_counter = merged_sorted_data[k][5]
tmp_array = []
frame_counter = 0

for k in range(len(merged_sorted_data)):
    if str(merged_sorted_data[k][3]) == "Sideline":
        if merged_sorted_data[k][5] > frame_counter :
            if frame_counter > 0:
                merged_sorted_reorganized_data_Sideline.append(tmp_array)
            tmp_array = []
            tmp_array.append(merged_sorted_data[k][0])
        tmp_array.append(np.concatenate((merged_sorted_data[k][5:10],merged_sorted_data[k][15:22])))
        frame_counter = merged_sorted_data[k][5]


In [ ]:
merged_sorted_reorganized_data_Sideline[0]

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

NFL_Field = cv2.imread("../input/nfl-football-field/1200px-AmFBfield.svg.png")

#renormalization factors for  x,y coordinates, based on NFL_Field png width and height

w_real = 120
h_real = 53.3
x_factor = NFL_Field.shape[1]/120
y_factor = NFL_Field.shape[0]/53.3



merged_sorted_reorganized_data = merged_sorted_reorganized_data_Endzone
merged_sorted_reorganized_data2 = merged_sorted_reorganized_data_Sideline

frame_idx = 0
vid = cv2.VideoCapture(f'{BASE_DIR}/train/'+str(merged_sorted_reorganized_data[frame_idx][0][0:-2]+".mp4"))
vid2 = cv2.VideoCapture(f'{BASE_DIR}/train/'+str(merged_sorted_reorganized_data2[frame_idx][0][0:-2]+".mp4"))
try:
    while(True):
                
        # Capture frame-by-frame
        ret, frame = vid.read()
        ret2, frame2 = vid2.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            break
            
        if not ret2:
            # Release the Video Device if ret is false
            vid2.release()
            # Message to be displayed after releasing the device
            break            
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        frame3 = cv2.cvtColor(NFL_Field, cv2.COLOR_BGR2RGB)
        
        # visualize helmet detection for endline camera 
        for helmets in range(1,len(merged_sorted_reorganized_data[frame_idx])):
            left = int(merged_sorted_reorganized_data[frame_idx][helmets][2]) 
            top = int(merged_sorted_reorganized_data[frame_idx][helmets][4])
            w,h = int(merged_sorted_reorganized_data[frame_idx][helmets][3]),int(merged_sorted_reorganized_data[frame_idx][helmets][5])
            start_point = (left,top)
            end_point = (left+w,top+h)
            color = (255,255,255)
            thickness = 5
            frame = cv2.rectangle(frame, start_point, end_point, color, thickness)
        
        # visualize helmet detection for sideline camera            
        for helmets in range(1,len(merged_sorted_reorganized_data2[frame_idx])):
            left2 = int(merged_sorted_reorganized_data2[frame_idx][helmets][2]) 
            top2 = int(merged_sorted_reorganized_data2[frame_idx][helmets][4])
            w2,h2 = int(merged_sorted_reorganized_data2[frame_idx][helmets][3]),int(merged_sorted_reorganized_data2[frame_idx][helmets][5])
            start_point2 = (left2,top2)
            end_point2 = (left2+w2,top2+h2)
            color = (255,255,255)
            thickness = 5
            frame2 = cv2.rectangle(frame2, start_point2, end_point2, color, thickness) 
        
        # visualize detection of NGS data         
        for coords in range(1,len(merged_sorted_reorganized_data2[frame_idx])):
            # invert y coordinates, as the origo of the y coordinates is not same as usual convention of x,y image plane... duhhh :D
            x = int(x_factor * (merged_sorted_reorganized_data2[frame_idx][coords][5]) ) 
            y = int(y_factor * (53.3-merged_sorted_reorganized_data2[frame_idx][coords][6]) )
            color = (255,0,0)
            thickness = 5
            frame3 = cv2.circle(frame3, (int(x),int(y) ),1, color, thickness) 
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        #imshow(frame)
        #imshow(frame2)
        columns = 3
        images = [frame,frame2,frame3]
        for i, image in enumerate(images):
            plt.subplot(len(images) / columns + 1, columns, i + 1)
            plt.imshow(image)
        
        show()

        # Display the frame until new frame is available
        time.sleep(0.05)
        clear_output(wait=True)
        
        frame_idx+=1
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device